In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [14]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [15]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['total']=0
vistas2=vistas
vistas['puntos']=0
vistas2=vistas2.groupby(['idAviso','idpostulante'],as_index=False).agg({'puntos':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,puntos
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [16]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'total':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,total
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [17]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [18]:
vistas2.head()

,idaviso,idpostulante,puntos,total
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [20]:
vistas2['vistas']=vistas2['puntos']
vistas2['puntos']=(vistas2['puntos']*10)/vistas2['total']
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,puntos,total,vistas
0,18,BolNL,0.714286,28,2
1,48375,RwVdKR,0.666667,15,1
2,169730,1KjXB,0.202020,99,2
3,169730,2AKzxa,10.000000,2,2
4,169730,6LJ64,0.434783,23,1


In [32]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/3.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/4.csv").iloc[0:len(postulaciones2)//2], ignore_index=True)
print(len(postulaciones2))
postulaciones2.head()

9906801


,idaviso,idpostulante,sepostulo
0,1112257047,ow2xPaj,0
1,1111920714,LNKXqJB,0
2,1112346945,8MBPqYD,0
3,1112345547,ek4P2Wr,0
4,1112237522,NpqMD5,0


In [33]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [34]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16450211


In [35]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,puntos,total,vistas
2714889,1112179207,dYjlNZM,0,FEM,18.0,-,-,-,-,En Curso,...,True,False,False,False,False,True,False,NaN,NaN,NaN
14119907,1112425062,8MPwakz,0,FEM,49.0,-,-,-,-,-,...,False,False,False,False,False,False,False,NaN,NaN,NaN
6054252,1112405351,N9r4Al,1,FEM,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
6308561,1112371208,YYEJar,0,MASC,44.0,En Curso,Graduado,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
2714699,1112179207,1LL9qb,0,FEM,37.0,-,-,-,-,Graduado,...,True,False,False,False,False,True,False,NaN,NaN,NaN
11032167,1112346459,GNZ2JOv,0,MASC,39.0,-,-,-,-,-,...,True,False,True,True,False,False,False,NaN,NaN,NaN
4022222,1111414856,8oG9L3,0,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,NaN,NaN,NaN
9892695,1112437516,DrzRL0Z,1,FEM,50.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,True,False,NaN,NaN,NaN
12147213,1112301407,MVr1X6o,1,FEM,28.0,-,-,-,-,-,...,True,False,False,False,False,False,False,NaN,NaN,NaN
1402338,1112305232,bOjr18b,0,MASC,19.0,-,-,-,-,-,...,False,False,False,False,False,False,False,NaN,NaN,NaN


In [38]:
postulaciones.puntos=postulaciones.puntos.fillna(0)
postulaciones.total=postulaciones.total.fillna(0)
postulaciones.vistas=postulaciones.vistas.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,puntos,total,vistas
2726979,1112394499,VNP22dE,1,FEM,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,0.066667,150.0,1.0
11262341,1112418587,0zkKkN9,0,MASC,43.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,0.000000,0.0,0.0
10799616,1112314232,DrJ3x8A,0,FEM,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,True,False,0.000000,0.0,0.0
6299035,1112268639,jkjlxkG,1,MASC,18.0,-,-,Graduado,-,Graduado,...,False,False,False,False,True,False,True,0.000000,0.0,0.0
13747415,1112289798,X95evjO,1,MASC,18.0,-,-,-,-,-,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
1864849,1112334791,Ezp3Y3J,1,FEM,23.0,-,-,-,-,Graduado,...,True,False,False,False,False,True,False,0.212766,47.0,1.0
6078339,1112429705,96z8Yea,0,MASC,23.0,-,-,-,-,-,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
10982394,1112313151,ZqD9lP,0,FEM,39.0,-,-,-,-,-,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
12726846,1112350617,BmBlJDE,1,MASC,20.0,-,-,-,-,-,...,False,False,False,False,False,True,False,0.833333,24.0,2.0
14454354,1112254360,6jo9BO,0,MASC,32.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,0.000000,0.0,0.0


In [39]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  14560399 Test:  766337


In [40]:
gc.collect()

175

In [41]:
c=lgb.LGBMClassifier(learning_rate=0.005,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [42]:
score=c.score(x_test,y_test)*100
print(score)

75.48558923815501


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [43]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV12.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV12.pkl']

In [45]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.puntos=prediccion.puntos.fillna(0)
prediccion.total=prediccion.total.fillna(0)
prediccion.vistas=prediccion.vistas.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,puntos,total,vistas
49861,3202,1111353878,ZvqVwE,MASC,46.0,-,-,-,Abandonado,-,...,True,False,False,False,False,False,False,0.000000,0.0,0.0
82879,31350,1112385711,6qOVZ4,MASC,35.0,-,Graduado,-,-,Graduado,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
39603,69731,1112461551,JBdPl6e,MASC,25.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
12366,4512,1111473085,dY92Vp0,FEM,19.0,-,-,-,-,Graduado,...,False,False,False,True,True,False,True,0.227273,88.0,2.0
12656,17687,1112205105,LNPkkQQ,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,1.212121,33.0,4.0
14239,62438,1112456089,DrlWNLd,FEM,25.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,0.094937,632.0,6.0
94789,94138,1112483253,ea0X9W,FEM,41.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
18761,76595,1112464229,owz0jPZ,MASC,31.0,-,-,-,-,-,...,False,False,False,False,False,False,False,0.000000,0.0,0.0
54579,35159,1112410012,JBmm8MO,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,0.146520,273.0,4.0
47718,88116,1112466977,LNMp3w5,MASC,31.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,0.042194,237.0,1.0


In [46]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.69800741, 0.30199259],
       [0.65576447, 0.34423553],
       [0.67069296, 0.32930704],
       ...,
       [0.86754174, 0.13245826],
       [0.6383185 , 0.3616815 ],
       [0.95734159, 0.04265841]])

In [47]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [48]:
prediccion['sepostulo']=y_final2

In [49]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,puntos,total,vistas,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.0,0.301993
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,0.0,0.0,0.0,0.344236
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.0,0.329307
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.0,0.114699
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,0.0,0.0,0.0,0.316637


In [50]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.301993
1,1,0.344236
2,2,0.329307
3,3,0.114699
4,4,0.316637


In [51]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion27.csv",index=False)

In [52]:
prediccion.sepostulo.mean()

0.5400703143864202